In [1]:
import pandas as pd
import numpy as np
import warnings 
import warnings
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

def load_data():
    #read the data
    df = pd.read_csv('SMSSpamCollection.txt', delimiter='\t',header=None)
    df.rename(columns = {0:'label',1: 'text'}, inplace = True)
    #Input and output variables
    X = df['text']
    y = df['label']
    return df, X, y

df, X, y = load_data()
df.text[5568]

'Will ü b going to esplanade fr home?'

In [2]:
from sklearn.utils import resample
dfSpam = df[df['label']=='spam']
dfHam = df[df['label']=='ham']

df1 = resample(dfHam, replace = True, n_samples = 747, random_state=42)
df = pd.concat([dfSpam, df1])
df['label'].value_counts()

spam    747
ham     747
Name: label, dtype: int64

In [3]:
X = df['text']
y = []
for i in df.label:
    if i == 'spam':
        y.append(1)
    else:
        y.append(0)

In [4]:

SEQUENCE_LENGTH = 100 # the length of all sequences (number of words per sample)
EMBEDDING_SIZE = 100  # Using 100-Dimensional GloVe embedding vectors

BATCH_SIZE = 64

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.pipeline import TransformerMixin
from sklearn.base import BaseEstimator

# Text tokenization
# vectorizing text, turning each text into sequence of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
# convert to sequence of integers
X = tokenizer.texts_to_sequences(X)
# convert to numpy arrays
X = np.array(X)
y = np.array(y)
# pad sequences at the beginning of each sequence with 0's
# for example if SEQUENCE_LENGTH=4:
# [[5, 3, 2], [5, 1, 2, 3], [3, 4]]
# will be transformed to:
# [[0, 5, 3, 2], [5, 1, 2, 3], [0, 0, 3, 4]]
X = pad_sequences(X, maxlen=SEQUENCE_LENGTH)

Using TensorFlow backend.


In [5]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional, LSTM

vocab_size = 20000  # Max number of different word, i.e. model input dimension
maxlen = 80  # Max number of words kept at the end of each text

batch_size = 64
max_features = vocab_size + 1

model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Bidirectional(LSTM(128, dropout=0.5, recurrent_dropout=0.5)))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

model.fit(X,y,epochs=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/5
1494/1494 [==============================] - 13s 9ms/step - loss: 0.5248 - accuracy: 0.7416
Epoch 2/5
1494/1494 [==============================] - 12s 8ms/step - loss: 0.1569 - accuracy: 0.9431
Epoch 3/5
1494/1494 [==============================] - 14s 9ms/step - loss: 0.0740 - accuracy: 0.9739
Epoch 4/5
1494/1494 [==============================] - 13s 8ms/step - loss: 0.0411 - accuracy: 0.9886
Epoch 5/5
1494/1494 [==============================] - 14s 9ms/step - loss: 0.0195 - accuracy: 0.9953


In [7]:
pred_text = ['Will ü b going to esplanade fr home?']
pred_text = tokenizer.texts_to_sequences(pred_text)
# convert to numpy arrays
pred_text = np.array(pred_text)

pred_text = pad_sequences(pred_text, maxlen=SEQUENCE_LENGTH)

model.predict_classes(pred_text)

array([[0]], dtype=int32)

In [8]:
model.predict_proba(pred_text)

array([[0.00410597]], dtype=float32)